In [1]:
import time
#to write data to an excel spreadsheet
#import pandas as pd
#import openpyxl
#The polynomial ring we are going to use
S.<ν> = QQ[]


#Defining a class of P objects
class P:
    def init(self, Integer_list, polynomial, max_degree):
        self.Integer_list = Integer_list

P_dict = {}

#need to reverse lists because Sage's "Partitions(n)" function makes lists the wrong way
def Reverse(lst):
    return [ele for ele in reversed(lst)]

#function to create P objects with partitions of any set size, the size gets multipled
#by 2 since partitions of odd numbers will always give 0.
#partitions are lists with the first element being the smallest last element is largest
#as a result of intentional reversing in the code
def create_lots_of_P(size):
    P_dict[(0,)] = P()
    P_dict[(0,)].Integer_list = [0]
    P_dict[(0,)].polynomial = ν
    P_dict[(0,)].max_degree = 1
    for i in range(1, size+1):
        for part in Partitions(2 *i):
            #reversing the lists that Partitions(size) is making
            Rev_part = Reverse(part)
            P_dict[tuple(Rev_part)] = P()
            P_dict[tuple(Rev_part)].Integer_list = Rev_part
            P_dict[tuple(Rev_part)].polynomial = -1
            P_dict[tuple(Rev_part)].max_degree = -1

#Function for generating just one polynomial without the use of a dictionary to remember terms.
#Slightly faster than using generate_polynomials if you are just looking for one polynomial
#because it only needs to call upon the partitions it creates as a result of the recursive step,
#thus no dictionary is needed
def gen_one_poly(lst):
    if lst == [0]:
        return ν

    if lst[0] == 0:
        del lst[0]
        return ν * gen_one_poly(lst)
    else:
        big_list_1 = []
        big_list_2 = []
        temp_list = []

        for r in range(1, lst[0]):
            temp_list.append(r - 1)
            temp_list.append(lst[0] - r - 1)

            for i in range(1, len(lst)):
                temp_list.append(lst[i])

            big_list_1.append(temp_list[:])
            temp_list.clear()

        for r in range(1, len(lst)):
            temp_list.append(lst[0] + lst[r] - 2)

            for i in range(1, len(lst)):
                temp_list.append(lst[i])

            del temp_list[r]
            big_list_2.append(temp_list[:])
            temp_list.clear()
        for i in range (len(big_list_1)):
            temp_list.append(gen_one_poly(sorted(big_list_1[i])))
        for i in range (len(big_list_2)):
            temp_list.append(lst[i+1] * gen_one_poly(sorted(big_list_2[i])))

        return sum(temp_list)

#Function that generates polynomials from integer lists using a dictionary to "remember" earlier terms.
#This is fast for generating lots and lots of polynomials
def generate_polynomial(lst):
    if tuple(lst) in P_dict:
        if P_dict[tuple(lst)].max_degree < len(lst):
            P_dict[tuple(lst)].max_degree = len(lst)
        if P_dict[tuple(lst)].polynomial != -1:
            return P_dict[tuple(lst)].polynomial
    if lst[0] == 0:
        del lst[0]
        return ν * generate_polynomial(lst)
    else:
        big_list_1 = []
        big_list_2 = []
        temp_list = []

        for r in range(1, lst[0]):
            temp_list.append(r - 1)
            temp_list.append(lst[0] - r - 1)

#             for i in range(1, len(lst)):
#                 temp_list.append(lst[i])
            temp_list += lst[1:]

            big_list_1.append(temp_list[:])
            temp_list.clear()

        for r in range(1, len(lst)):
            temp_list.append(lst[0] + lst[r] - 2)

#             for i in range(1, len(lst)):
#                 temp_list.append(lst[i])
            temp_list += lst[1:]

            del temp_list[r]
            big_list_2.append(temp_list[:])
            temp_list.clear()
        for i in range (len(big_list_1)):
            temp_list.append(generate_polynomial(sorted(big_list_1[i])))
        for i in range (len(big_list_2)):
            temp_list.append(lst[i+1] * generate_polynomial(sorted(big_list_2[i])))

        return sum(temp_list)

#function to assign polynomials to each P
def assign_polynomials():
    for key in P_dict:
        P_dict[key].polynomial = generate_polynomial(P_dict[key].Integer_list)

#function to generate pattern partitions via appending appendPart to startPart
def pattern_partitions(startPart,appendPart,maxSize):
    storageList=[]
    storageList.append(startPart[:])
    fixPart=startPart
    for i in range (0,maxSize):
        fixPart+=appendPart
        fixPart.sort()
        storageList.append(fixPart[:])
    return storageList

#function to turn list of lists into list
def lol_to_list(lol):
    lol_lst=[]
    for lst in lol:
        for i in lst:
            lol_lst.append(i)
    return lol_lst

#funtion to generate sequential partitions
def seq_partitions(fix_list,var_list,increm,limit):
    lst=[fix_list,var_list]
    seq_part_list=[]
    cpy_lst=lst[:]
    cpy_var_lst=var_list[:]
    for i in range(0,limit):
        append_lst=lol_to_list(cpy_lst)
        srtd_append_lst=sorted(append_lst,reverse=false)
        seq_part_list.append(srtd_append_lst)
        for j in range (0,len(var_list)):
            cpy_var_lst[j]+=increm
        cpy_lst[1]=cpy_var_lst
    return seq_part_list
    
    
    
#function for telling us the results
def all_polys():
    print('{0:<50} {1:<}'.format('Partition', 'Polynomial')) 
    for key in P_dict:
        print('{0:<50} {1:<}'.format(str(P_dict[key].Integer_list), str(P_dict[key].polynomial)))

#takes in a list of lists and only prints the polynomials of those partitions you specificy
def some_polys(lst):
    print('{0:<50} {1:<}'.format('Partition', 'Polynomial'))
    for l in lst:
        print('{0:<50} {1:<}'.format(str(P_dict[tuple(l)].Integer_list), str(P_dict[tuple(l)].polynomial)))

def all_factors():
    print('{0:<50} {1:<}'.format('Partition', 'Factorization'))
    for key in P_dict:
        print('{0:<50} {1:<}'.format(str(P_dict[key].Integer_list), str(P_dict[key].polynomial.factor())))

def some_factors(lst):
    print('{0:<50} {1:<}'.format('Partition', 'Factorization'))
    for l in lst:
        print('{0:<50} {1:<}'.format(str(P_dict[tuple(l)].Integer_list), str(P_dict[tuple(l)].polynomial.factor())))

def all_results():
    print('{0:<40} {1:<80} {2:<}'.format('Partition', 'Polynomial', 'Factorization'))
    for key in P_dict:
        print('{0:<40} {1:<80} {2:<}'.format(str(P_dict[key].Integer_list), str(P_dict[key].polynomial), str(P_dict[key].polynomial.factor())))

def some_results(lst):
    print('{0:<40} {1:<80} {2:<}'.format('Partition', 'Polynomials', 'Factorization'))
    for l in lst:
        print('{0:<40} {1:<80} {2:<}'.format(str(P_dict[tuple(l)].Integer_list), str(P_dict[tuple(l)].polynomial), str(P_dict[tuple(l)].polynomial.factor())))

def all_max_degrees():
    print('{0:<50} {1:<}'.format('Partition', 'Max degree'))
    for key in P_dict:
        print('{0:<50} {1:<}'.format(str(P_dict[key].Integer_list), P_dict[key].max_degree))

def B_maker(length):
    b_list = []
    b_list.append(ν^0)
    for i in range(1,length):
        b_list.append(b_list[i-1] * ((ν - (i-1)) / i))
    return b_list
def zero_appender(n,lst):
    for i in range (n - len(lst)):
        lst.append(0)
    return lst
def mono_to_binom(poly):
    n = poly.degree() + 1 #size of basis change matrix
    Mono_to_Binom_Mat=Matrix(QQ,n,n,lambda i, j:zero_appender(n,B_maker(n)[j].coefficients(sparse=false))[i]).inverse()
    #^^^basis change matrix for deg(poly)^^^
    binom_coefficients = Mono_to_Binom_Mat * vector(poly.coefficients(sparse = false))#vector of CoB*poly
    return binom_coefficients #return vector as tuple i.e. immutable

def all_binoms():
    print('{0:<50} {1:<}'.format('Partition', 'Binomial coefficients'))
    for key in P_dict:
        print('{0:<50} {1:<}'.format(str(P_dict[key].Integer_list), str(mono_to_binom(P_dict[key].polynomial))))
        
def some_binoms(lst):
    print('{0:<50} {1:<}'.format('Partition', 'Binomial coefficients'))
    for l in lst:
        print('{0:<50} {1:<}'.format(str(P_dict[tuple(l)].Integer_list), str(mono_to_binom(P_dict[tuple(l)].polynomial))))
def div_all_binoms():
    print('{0:<50} {1:<}'.format('Partition', 'Binomial coefficients divided out'))
    for key in P_dict:
        m = mono_to_binom(P_dict[key].polynomial)
        print('{0:<50} {1:<}'.format(str(P_dict[key].Integer_list), str(m / m[1])))

def div_some_binoms(lst):
    print('{0:<50} {1:<}'.format('Partition', 'Binomial coefficients divided out'))
    for l in lst:
        m = mono_to_binom(P_dict[tuple(l)].polynomial)
        print('{0:<50} {1:<}'.format(str(P_dict[tuple(l)].Integer_list), str(m / m[1])))
        
def find_galois(poly): #method for finding galois group of polynomial, needs work to reduce polys s.t. lowest term is constant
    galois_lst=[]
    E.<x>=NumberField(poly) #some galois ext. for poly bc rationals
    G=E.galois_group()
    galois_gens=G.gens()
    return galois_gens
def all_galois():
    print('{0:<50} {1:<}'.format('Partition', 'Galois Group'))
    for key in P_dict:
        print('{0:<50} {1:<}'.format(str(P_dict[key].Integer_list), str(find_galois(P_dict[key].polynomial))))
create_lots_of_P(12)
assign_polynomials()
List1=pattern_partitions([1,1],[1,1],9)
List2=seq_partitions([1],[1],2,14)
#div_some_binoms(List2)
#some_binoms(List2)
all_polys()

Partition                                          Polynomial
[0]                                                ν
[2]                                                ν^2
[1, 1]                                             ν
[4]                                                2*ν^3 + ν
[1, 3]                                             3*ν^2
[2, 2]                                             ν^4 + 2*ν^2
[1, 1, 2]                                          ν^3 + 2*ν
[1, 1, 1, 1]                                       3*ν^2
[6]                                                5*ν^4 + 10*ν^2
[1, 5]                                             10*ν^3 + 5*ν
[2, 4]                                             2*ν^5 + 9*ν^3 + 4*ν
[1, 1, 4]                                          2*ν^4 + 13*ν^2
[3, 3]                                             12*ν^3 + 3*ν
[1, 2, 3]                                          3*ν^4 + 12*ν^2
[1, 1, 1, 3]                                       9*ν^3 + 6*ν
[2, 2, 2]         

[1, 1, 6, 8]                                       70*ν^10 + 5130*ν^8 + 162225*ν^6 + 971810*ν^4 + 887790*ν^2
[3, 5, 8]                                          630*ν^9 + 54390*ν^7 + 576345*ν^5 + 1173900*ν^3 + 221760*ν
[1, 2, 5, 8]                                       140*ν^10 + 11130*ν^8 + 191940*ν^6 + 952105*ν^4 + 871710*ν^2
[1, 1, 1, 5, 8]                                    420*ν^9 + 36750*ν^7 + 484680*ν^5 + 1238055*ν^3 + 267120*ν
[4, 4, 8]                                          56*ν^11 + 3080*ν^9 + 64218*ν^7 + 564550*ν^5 + 1173361*ν^3 + 221760*ν
[1, 3, 4, 8]                                       84*ν^10 + 6510*ν^8 + 176232*ν^6 + 1002015*ν^4 + 842184*ν^2
[2, 2, 4, 8]                                       28*ν^12 + 1442*ν^10 + 28140*ν^8 + 251405*ν^6 + 942802*ν^4 + 803208*ν^2
[1, 1, 2, 4, 8]                                    28*ν^11 + 1834*ν^9 + 47460*ν^7 + 468041*ν^5 + 1227422*ν^3 + 282240*ν
[1, 1, 1, 1, 4, 8]                                 84*ν^10 + 6510*ν^8 + 176232*ν^6 + 10020

[1, 1, 1, 2, 2, 2, 2, 7]                           105*ν^13 + 6090*ν^11 + 138390*ν^9 + 1538880*ν^7 + 8405880*ν^5 + 18725280*ν^3 + 5644800*ν
[1, 1, 1, 1, 1, 2, 2, 2, 7]                        525*ν^12 + 27300*ν^10 + 531720*ν^8 + 4671240*ν^6 + 16810080*ν^4 + 12418560*ν^2
[1, 1, 1, 1, 1, 1, 1, 2, 2, 7]                     3675*ν^11 + 161700*ν^9 + 2441670*ν^7 + 13778940*ν^5 + 16944480*ν^3 + 1128960*ν
[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 7]                  33075*ν^10 + 1124550*ν^8 + 10742760*ν^6 + 19656000*ν^4 + 2903040*ν^2
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7]               363825*ν^9 + 8004150*ν^7 + 21101850*ν^5 + 4989600*ν^3
[6, 6, 6]                                          125*ν^12 + 9750*ν^10 + 307500*ν^8 + 4392550*ν^6 + 17433900*ν^4 + 12315600*ν^2
[1, 5, 6, 6]                                       250*ν^11 + 25125*ν^9 + 1018500*ν^7 + 9725900*ν^5 + 19808850*ν^3 + 3880800*ν
[2, 4, 6, 6]                                       50*ν^13 + 3665*ν^11 + 108420*ν^9 + 1569280*ν^7 + 10257250*ν^5 + 1893

[1, 1, 1, 1, 2, 2, 2, 2, 8]                        42*ν^15 + 4410*ν^13 + 176463*ν^11 + 3495660*ν^9 + 36170148*ν^7 + 183232560*ν^5 + 350364672*ν^3 + 81285120*ν
[1, 1, 1, 1, 1, 1, 2, 2, 2, 8]                     210*ν^14 + 23730*ν^12 + 891555*ν^10 + 14964600*ν^8 + 115213140*ν^6 + 340744320*ν^4 + 182891520*ν^2
[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 8]                  1470*ν^13 + 174930*ν^11 + 5614665*ν^9 + 68965050*ν^7 + 302813280*ν^5 + 265547520*ν^3 + 11612160*ν
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 8]               13230*ν^12 + 1627290*ν^10 + 38254545*ν^8 + 256490010*ν^6 + 325684800*ν^4 + 32659200*ν^2
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8]            145530*ν^11 + 18191250*ν^9 + 209781495*ν^7 + 366735600*ν^5 + 59875200*ν^3
[6, 7, 7]                                          14000*ν^11 + 1375500*ν^9 + 31031700*ν^7 + 211018675*ν^5 + 351952650*ν^3 + 59336550*ν
[1, 5, 7, 7]                                       77000*ν^10 + 5502000*ν^8 + 83444550*ν^6 + 331583350*ν^4 + 234122175*ν^2
[2, 4, 7, 7]       

[1, 1, 1, 1, 1, 2, 2, 2, 3, 6]                     225*ν^14 + 22350*ν^12 + 793500*ν^10 + 13013160*ν^8 + 100606080*ν^6 + 313856640*ν^4 + 226437120*ν^2
[1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 6]                  1575*ν^13 + 154350*ν^11 + 4652550*ν^9 + 56332080*ν^7 + 258468840*ν^5 + 308992320*ν^3 + 26127360*ν
[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 6]               14175*ν^12 + 1341900*ν^10 + 29786400*ν^8 + 203553000*ν^6 + 354715200*ν^4 + 65318400*ν^2
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 6]            155925*ν^11 + 13929300*ν^9 + 154781550*ν^7 + 376091100*ν^5 + 109771200*ν^3
[2, 2, 2, 2, 2, 2, 2, 6]                           5*ν^18 + 430*ν^16 + 15680*ν^14 + 315280*ν^12 + 3800720*ν^10 + 27889120*ν^8 + 120021120*ν^6 + 270443520*ν^4 + 232243200*ν^2
[1, 1, 2, 2, 2, 2, 2, 2, 6]                        5*ν^17 + 460*ν^15 + 17990*ν^13 + 387940*ν^11 + 4989320*ν^9 + 38421040*ν^7 + 166032960*ν^5 + 328757760*ν^3 + 116121600*ν
[1, 1, 1, 1, 2, 2, 2, 2, 2, 6]                     15*ν^16 + 1440*ν^14 + 56940*ν^12 + 11934

...

[1, 1, 1, 2, 2, 2, 3, 5, 7]                        22365*ν^15 + 2272200*ν^13 + 91825755*ν^11 + 1865459925*ν^9 + 19654352760*ν^7 + 97663950300*ν^5 + 164404109520*ν^3 + 32552150400*ν
[1, 1, 1, 1, 1, 2, 2, 3, 5, 7]                     143325*ν^14 + 13068300*ν^12 + 453037935*ν^10 + 7383251295*ν^8 + 55688906490*ν^6 + 157264646280*ν^4 + 95431089600*ν^2
[1, 1, 1, 1, 1, 1, 1, 2, 3, 5, 7]                  1223775*ν^13 + 95983650*ν^11 + 2644008885*ν^9 + 30254213745*ν^7 + 127594128690*ν^5 + 140276616480*ν^3 + 15367968000*ν
[1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 5, 7]               12998475*ν^12 + 825485850*ν^10 + 15706040805*ν^8 + 96362193375*ν^6 + 160513027920*ν^4 + 42814396800*ν^2
[2, 2, 2, 2, 2, 2, 5, 7]                           700*ν^18 + 76300*ν^16 + 3514595*ν^14 + 88675090*ν^12 + 1325099300*ν^10 + 11790515800*ν^8 + 59036380480*ν^6 + 141909317760*ν^4 + 102080563200*ν^2
[1, 1, 2, 2, 2, 2, 2, 5, 7]                        1400*ν^17 + 153650*ν^15 + 7081795*ν^13 + 176866340*ν^11 + 2566912600*ν^9 + 21410

[1, 3, 4, 4, 6, 6]                                 300*ν^16 + 50280*ν^14 + 3938271*ν^12 + 181507740*ν^10 + 4728323028*ν^8 + 50400138270*ν^6 + 162515993976*ν^4 + 98404191360*ν^2
[2, 2, 4, 4, 6, 6]                                 100*ν^18 + 13760*ν^16 + 833717*ν^14 + 28904454*ν^12 + 617429236*ν^10 + 7978799842*ν^8 + 55643429492*ν^6 + 156116702544*ν^4 + 95848030080*ν^2
[1, 1, 2, 4, 4, 6, 6]                              100*ν^17 + 15360*ν^15 + 1090397*ν^13 + 46551914*ν^11 + 1232057256*ν^9 + 17415753922*ν^7 + 102423701172*ν^5 + 166694211504*ν^3 + 28420761600*ν
[1, 1, 1, 1, 4, 4, 6, 6]                           300*ν^16 + 50280*ν^14 + 3938271*ν^12 + 181507740*ν^10 + 4728323028*ν^8 + 50400138270*ν^6 + 162515993976*ν^4 + 98404191360*ν^2
[2, 3, 3, 4, 6, 6]                                 600*ν^16 + 101730*ν^14 + 7772595*ν^12 + 318096390*ν^10 + 6363256380*ν^8 + 55172678430*ν^6 + 160800486300*ν^4 + 93571750800*ν^2
[1, 1, 3, 3, 4, 6, 6]                              1500*ν^15 + 269700*ν^13 + 216168

[1, 1, 1, 1, 1, 1, 4, 4, 4, 6]                     600*ν^16 + 98220*ν^14 + 7245270*ν^12 + 300869895*ν^10 + 6800836830*ν^8 + 61551168570*ν^6 + 164827123200*ν^4 + 82746800640*ν^2
[1, 3, 3, 3, 4, 4, 6]                              2160*ν^15 + 360828*ν^13 + 25795044*ν^11 + 957554379*ν^9 + 17299640772*ν^7 + 107743754718*ν^5 + 164640324924*ν^3 + 25566710400*ν
[2, 2, 3, 3, 4, 4, 6]                              240*ν^17 + 37212*ν^15 + 2541684*ν^13 + 95868771*ν^11 + 2021639592*ν^9 + 22275989682*ν^7 + 108695760684*ν^5 + 157914887760*ν^3 + 25227417600*ν
[1, 1, 2, 3, 3, 4, 4, 6]                           600*ν^16 + 96780*ν^14 + 6786870*ν^12 + 255463275*ν^10 + 5092071360*ν^8 + 48087291540*ν^6 + 159118809120*ν^4 + 103673623680*ν^2
[1, 1, 1, 1, 3, 3, 4, 4, 6]                        2880*ν^15 + 474804*ν^13 + 32926752*ν^11 + 1144801737*ν^9 + 18156704166*ν^7 + 106164391734*ν^5 + 164150252352*ν^3 + 26584588800*ν
[1, 2, 2, 2, 3, 4, 4, 6]                           60*ν^18 + 8928*ν^16 + 594195*ν^14 + 227077

[1, 3, 3, 3, 4, 5, 5]                              176580*ν^13 + 20457090*ν^11 + 905385060*ν^9 + 17320433580*ν^7 + 107724913335*ν^5 + 164491493580*ν^3 + 25771284000*ν
[2, 2, 3, 3, 4, 5, 5]                              12420*ν^15 + 1549890*ν^13 + 77882040*ν^11 + 1893684330*ν^9 + 22184129115*ν^7 + 108849552030*ν^5 + 157730477400*ν^3 + 25496856000*ν
[1, 1, 2, 3, 3, 4, 5, 5]                           45300*ν^14 + 5258850*ν^12 + 238149300*ν^10 + 5050365000*ν^8 + 48228935025*ν^6 + 158741906550*ν^4 + 103969483200*ν^2
[1, 1, 1, 1, 3, 3, 4, 5, 5]                        277740*ν^13 + 29347470*ν^11 + 1127944980*ν^9 + 18146305890*ν^7 + 106194185505*ν^5 + 164141513640*ν^3 + 26594568000*ν
[1, 2, 2, 2, 3, 4, 5, 5]                           2880*ν^16 + 356100*ν^14 + 18171240*ν^12 + 483087375*ν^10 + 6964074570*ν^8 + 51046074300*ν^6 + 153027025560*ν^4 + 104695351200*ν^2
[1, 1, 1, 2, 2, 3, 4, 5, 5]                        12240*ν^15 + 1436580*ν^13 + 67330980*ν^11 + 1571679135*ν^9 + 18616157730*ν^7 + 99839

[2, 2, 2, 3, 3, 3, 3, 3, 3]                        25920*ν^15 + 2507760*ν^13 + 99006300*ν^11 + 2015257725*ν^9 + 21700789380*ν^7 + 109536408540*ν^5 + 158630691600*ν^3 + 24249456000*ν
[1, 1, 2, 2, 3, 3, 3, 3, 3, 3]                     142560*ν^14 + 11664000*ν^12 + 376441830*ν^10 + 5994387585*ν^8 + 47401006410*ν^6 + 157023104040*ν^4 + 105427396800*ν^2
[1, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3]                  1127520*ν^13 + 73823400*ν^11 + 1814451840*ν^9 + 20460669255*ν^7 + 101256260490*ν^5 + 163528463520*ν^3 + 29099347200*ν
[1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3]               11411280*ν^12 + 539605800*ν^10 + 9019880550*ν^8 + 62246983275*ν^6 + 155669551920*ν^4 + 88746710400*ν^2
[1, 2, 2, 2, 2, 3, 3, 3, 3, 3]                     6480*ν^16 + 651240*ν^14 + 27077085*ν^12 + 597504600*ν^10 + 7422177420*ν^8 + 50039156160*ν^6 + 154067365440*ν^4 + 104080204800*ν^2
[1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3]                  38880*ν^15 + 3410100*ν^13 + 120088575*ν^11 + 2153696850*ν^9 + 20459257020*ν^7 + 95421353400*ν^

In [3]:
#The polynomial ring we are going to use
S.<ν> = QQ[]

#Defining a class of P objects
class P:
    def init(self, Integer_list, polynomial):
        self.Integer_list = Integer_list

P_dict = {}

#need to reverse lists because Sage's "Partitions(n)" function makes lists the wrong way
def Reverse(lst):
    return [ele for ele in reversed(lst)]

#function to create P objects with partitions of any set size
#SIZE MUST BE EVEN
#partitions are lists with the first element being the smallest last element is largest
def create_lots_of_P(size):
    P_dict[(0,)] = P()
    P_dict[(0,)].Integer_list = [0]
    P_dict[(0,)].polynomial = ν
    for i in range(1, size+1):
        for part in Partitions(2 *i):
            #reversing the lists that Partitions(size) is making
            Rev_part = Reverse(part)
            P_dict[tuple(Rev_part)] = P()
            P_dict[tuple(Rev_part)].Integer_list = Rev_part
            P_dict[tuple(Rev_part)].polynomial = -1


#function that generates polynomials from integer lists
def generate_polynomial(lst):
    if tuple(lst) in P_dict:
        if P_dict[tuple(lst)].polynomial != -1:
            return P_dict[tuple(lst)].polynomial
    if lst[0] == 0:
        del lst[0]
        return ν * generate_polynomial(lst)
    else:
        big_list_1 = []
        big_list_2 = []
        temp_list = []

        for r in range(1, lst[0]):
            temp_list.append(r - 1)
            temp_list.append(lst[0] - r - 1)

            for i in range(1, len(lst)):
                temp_list.append(lst[i])

            big_list_1.append(temp_list[:])
            temp_list.clear()

        for r in range(1, len(lst)):
            temp_list.append(lst[0] + lst[r] - 2)

            for i in range(1, len(lst)):
                temp_list.append(lst[i])

            del temp_list[r]
            big_list_2.append(temp_list[:])
            temp_list.clear()
        if big_list_1:
            for i in range (len(big_list_1)):
                temp_list.append(generate_polynomial(sorted(big_list_1[i])))
        if big_list_2:
            for i in range (len(big_list_2)):
                temp_list.append(lst[i+1] * generate_polynomial(sorted(big_list_2[i])))

        return sum(temp_list)

#function to assign polynomials to each P
def assign_polynomials():
    for key in P_dict:
        P_dict[key].polynomial = generate_polynomial(P_dict[key].Integer_list)


#function for telling us the results
def results():
    for key in P_dict:
        print(P_dict[key].Integer_list, end = ": ")
        print(P_dict[key].polynomial)





create_lots_of_P(15)
assign_polynomials()



In [19]:
import time
print("hi")
#The polynomial ring we are going to use
S.<ν> = QQ[]

#Defining a class of P objects
class P:
    def init(self, Integer_list, polynomial):
        self.Integer_list = Integer_list

P_dict = {}

#need to reverse lists because Sage's "Partitions(n)" function makes lists the wrong way
def Reverse(lst):
    return [ele for ele in reversed(lst)]

#function to create P objects with partitions of any set size
#SIZE MUST BE EVEN
#partitions are lists with the first element being the smallest last element is largest
def create_lots_of_P(size):
    P_dict[(0,)] = P()
    P_dict[(0,)].Integer_list = [0]
    P_dict[(0,)].polynomial = ν
    for i in range(1, size+1):
        for part in Partitions(2 *i):
            #reversing the lists that Partitions(size) is making
            Rev_part = Reverse(part)
            P_dict[tuple(Rev_part)] = P()
            P_dict[tuple(Rev_part)].Integer_list = Rev_part
            P_dict[tuple(Rev_part)].polynomial = -1


#function that generates polynomials from integer lists
def generate_polynomial(lst):
    if tuple(lst) in P_dict:
        if P_dict[tuple(lst)].polynomial != -1:
            return P_dict[tuple(lst)].polynomial
    if lst[0] == 0:
        del lst[0]
        return ν * generate_polynomial(lst)
    else:
        big_list_1 = []
        big_list_2 = []
        temp_list = []

        for r in range(1, lst[0]):
            temp_list.append(r - 1)
            temp_list.append(lst[0] - r - 1)

            for i in range(1, len(lst)):
                temp_list.append(lst[i])

            big_list_1.append(temp_list[:])
            temp_list.clear()

        for r in range(1, len(lst)):
            temp_list.append(lst[0] + lst[r] - 2)

            for i in range(1, len(lst)):
                temp_list.append(lst[i])

            del temp_list[r]
            big_list_2.append(temp_list[:])
            temp_list.clear()
        for i in range (len(big_list_1)):
            temp_list.append(generate_polynomial(sorted(big_list_1[i])))
        for i in range (len(big_list_2)):
            temp_list.append(lst[i+1] * generate_polynomial(sorted(big_list_2[i])))

        return sum(temp_list)

#function to assign polynomials to each P
def assign_polynomials():
    for key in P_dict:
        P_dict[key].polynomial = generate_polynomial(P_dict[key].Integer_list)


#function for telling us the results
def results():
    for key in P_dict:
        print(P_dict[key].Integer_list, end = ": ")
        print(P_dict[key].polynomial)





#start_time = time.time()
create_lots_of_P(10)
assign_polynomials()
results()
#print("--- %s seconds ---" % (time.time() - start_time))
print("hi")
gen_one_poly([1,1])


hi
[0]: ν
[2]: ν^2
[1, 1]: ν
[4]: 2*ν^3 + ν
[1, 3]: 3*ν^2
[2, 2]: ν^4 + 2*ν^2
[1, 1, 2]: ν^3 + 2*ν
[1, 1, 1, 1]: 3*ν^2
[6]: 5*ν^4 + 10*ν^2
[1, 5]: 10*ν^3 + 5*ν
[2, 4]: 2*ν^5 + 9*ν^3 + 4*ν
[1, 1, 4]: 2*ν^4 + 13*ν^2
[3, 3]: 12*ν^3 + 3*ν
[1, 2, 3]: 3*ν^4 + 12*ν^2
[1, 1, 1, 3]: 9*ν^3 + 6*ν
[2, 2, 2]: ν^6 + 6*ν^4 + 8*ν^2
[1, 1, 2, 2]: ν^5 + 6*ν^3 + 8*ν
[1, 1, 1, 1, 2]: 3*ν^4 + 12*ν^2
[1, 1, 1, 1, 1, 1]: 15*ν^3
[8]: 14*ν^5 + 70*ν^3 + 21*ν
[1, 7]: 35*ν^4 + 70*ν^2
[2, 6]: 5*ν^6 + 40*ν^4 + 60*ν^2
[1, 1, 6]: 5*ν^5 + 70*ν^3 + 30*ν
[3, 5]: 45*ν^4 + 60*ν^2
[1, 2, 5]: 10*ν^5 + 65*ν^3 + 30*ν
[1, 1, 1, 5]: 30*ν^4 + 75*ν^2
[4, 4]: 4*ν^6 + 40*ν^4 + 61*ν^2
[1, 3, 4]: 6*ν^5 + 75*ν^3 + 24*ν
[2, 2, 4]: 2*ν^7 + 21*ν^5 + 58*ν^3 + 24*ν
[1, 1, 2, 4]: 2*ν^6 + 25*ν^4 + 78*ν^2
[1, 1, 1, 1, 4]: 6*ν^5 + 75*ν^3 + 24*ν
[2, 3, 3]: 12*ν^5 + 75*ν^3 + 18*ν
[1, 1, 3, 3]: 30*ν^4 + 75*ν^2
[1, 2, 2, 3]: 3*ν^6 + 30*ν^4 + 72*ν^2
[1, 1, 1, 2, 3]: 9*ν^5 + 60*ν^3 + 36*ν
[1, 1, 1, 1, 1, 3]: 45*ν^4 + 60*ν^2
[2, 2, 2, 2]: ν^8 + 12*ν^

[1, 1, 2, 2, 3, 7]: 378*ν^10 + 14238*ν^8 + 183771*ν^6 + 886662*ν^4 + 941976*ν^2
[1, 1, 1, 1, 2, 3, 7]: 1764*ν^9 + 55146*ν^7 + 509061*ν^5 + 1178814*ν^3 + 282240*ν
[1, 1, 1, 1, 1, 1, 3, 7]: 11970*ν^8 + 267750*ν^6 + 1087065*ν^4 + 660240*ν^2
[1, 2, 2, 2, 2, 7]: 35*ν^12 + 1610*ν^10 + 28140*ν^8 + 228760*ν^6 + 827680*ν^4 + 940800*ν^2
[1, 1, 1, 2, 2, 2, 7]: 105*ν^11 + 4410*ν^9 + 67830*ν^7 + 453600*ν^5 + 1148280*ν^3 + 352800*ν
[1, 1, 1, 1, 1, 2, 2, 7]: 525*ν^10 + 18900*ν^8 + 229320*ν^6 + 1002120*ν^4 + 776160*ν^2
[1, 1, 1, 1, 1, 1, 1, 2, 7]: 3675*ν^9 + 102900*ν^7 + 795270*ν^5 + 1054620*ν^3 + 70560*ν
[1, 1, 1, 1, 1, 1, 1, 1, 1, 7]: 33075*ν^8 + 595350*ν^6 + 1217160*ν^4 + 181440*ν^2
[4, 6, 6]: 50*ν^11 + 2865*ν^9 + 62580*ν^7 + 568000*ν^5 + 1169250*ν^3 + 224280*ν
[1, 3, 6, 6]: 75*ν^10 + 5700*ν^8 + 178500*ν^6 + 997110*ν^4 + 845640*ν^2
[2, 2, 6, 6]: 25*ν^12 + 1350*ν^10 + 27300*ν^8 + 249770*ν^6 + 947220*ν^4 + 801360*ν^2
[1, 1, 2, 6, 6]: 25*ν^11 + 1650*ν^9 + 46200*ν^7 + 476570*ν^5 + 1205220*ν^3 + 297360*

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]: 2027025*ν^8
[18]: 4862*ν^10 + 291720*ν^8 + 4390386*ν^6 + 17454580*ν^4 + 12317877*ν^2
[1, 17]: 24310*ν^9 + 1021020*ν^7 + 9699690*ν^5 + 19885580*ν^3 + 3828825*ν
[2, 16]: 1430*ν^11 + 82940*ν^9 + 1531530*ν^7 + 10298860*ν^5 + 18941065*ν^3 + 3603600*ν
[1, 1, 16]: 1430*ν^10 + 163020*ν^8 + 3453450*ν^6 + 17025580*ν^4 + 13815945*ν^2
[3, 15]: 34320*ν^9 + 1261260*ν^7 + 10540530*ν^5 + 19244940*ν^3 + 3378375*ν
[1, 2, 15]: 6435*ν^10 + 283140*ν^8 + 3873870*ν^6 + 16705260*ν^4 + 13590720*ν^2
[1, 1, 1, 15]: 19305*ν^9 + 900900*ν^7 + 9279270*ν^5 + 20205900*ν^3 + 4054050*ν
[4, 14]: 858*ν^11 + 56485*ν^9 + 1345344*ν^7 + 10437570*ν^5 + 19195748*ν^3 + 3423420*ν
[1, 3, 14]: 1287*ν^10 + 180180*ν^8 + 3729726*ν^6 + 17323020*ν^4 + 13225212*ν^2
[2, 2, 14]: 429*ν^12 + 24882*ν^10 + 522522*ν^8 + 4729296*ν^6 + 16497624*ν^4 + 12684672*ν^2
[1, 1, 2, 14]: 429*ν^11 + 42900*ν^9 + 1063062*ν^7 + 9017580*ν^5 + 20011134*ν^3 + 4324320*ν
[1, 1, 1, 1, 14]: 1287*ν^10 + 180180*ν^8 + 37

[1, 2, 5, 5, 5]: 5400*ν^10 + 269100*ν^8 + 3909150*ν^6 + 16628175*ν^4 + 13647600*ν^2
[1, 1, 1, 5, 5, 5]: 22200*ν^9 + 913500*ν^7 + 9226350*ν^5 + 20328375*ν^3 + 3969000*ν
[4, 4, 5, 5]: 720*ν^11 + 52800*ν^9 + 1353240*ν^7 + 10422000*ν^5 + 19203465*ν^3 + 3427200*ν
[1, 3, 4, 5, 5]: 2880*ν^10 + 183300*ν^8 + 3728760*ν^6 + 17325375*ν^4 + 13219110*ν^2
[2, 2, 4, 5, 5]: 360*ν^12 + 22980*ν^10 + 511770*ν^8 + 4757085*ν^6 + 16429470*ν^4 + 12737760*ν^2
[1, 1, 2, 4, 5, 5]: 760*ν^11 + 48700*ν^9 + 1068270*ν^7 + 8979575*ν^5 + 20128520*ν^3 + 4233600*ν
[1, 1, 1, 1, 4, 5, 5]: 3480*ν^10 + 206700*ν^8 + 3668910*ν^6 + 17450775*ν^4 + 13129560*ν^2
[2, 3, 3, 5, 5]: 6210*ν^10 + 312300*ν^8 + 4444020*ν^6 + 17362575*ν^4 + 12334320*ν^2
[1, 1, 3, 3, 5, 5]: 22650*ν^9 + 896700*ν^7 + 9297750*ν^5 + 20172525*ν^3 + 4069800*ν
[1, 2, 2, 3, 5, 5]: 1440*ν^11 + 73650*ν^9 + 1319955*ν^7 + 9357000*ν^5 + 19372980*ν^3 + 4334400*ν
[1, 1, 1, 2, 3, 5, 5]: 6120*ν^10 + 262350*ν^8 + 3597615*ν^6 + 16357500*ν^4 + 14235840*ν^2
[1, 1, 1, 1, 1, 3, 5

[1, 1, 1, 1, 4, 12]: 792*ν^12 + 115236*ν^10 + 5454306*ν^8 + 82861218*ν^6 + 333696627*ν^4 + 232600896*ν^2
[2, 3, 3, 12]: 1584*ν^12 + 256212*ν^10 + 8391042*ν^8 + 95064156*ν^6 + 332541099*ν^4 + 218474982*ν^2
[1, 1, 3, 3, 12]: 3960*ν^11 + 744480*ν^9 + 22820490*ν^7 + 191963970*ν^5 + 370589175*ν^3 + 68607000*ν
[1, 2, 2, 3, 12]: 396*ν^13 + 53658*ν^11 + 1999008*ν^9 + 30418839*ν^7 + 190970406*ν^5 + 359436528*ν^3 + 71850240*ν
[1, 1, 1, 2, 3, 12]: 1188*ν^12 + 192654*ν^10 + 6581124*ν^8 + 79877457*ν^6 + 318236688*ν^4 + 249839964*ν^2
[1, 1, 1, 1, 1, 3, 12]: 5940*ν^11 + 1109790*ν^9 + 30034620*ν^7 + 212512905*ν^5 + 352188540*ν^3 + 58877280*ν
[2, 2, 2, 2, 12]: 132*ν^15 + 10230*ν^13 + 321948*ν^11 + 5227365*ν^9 + 45634908*ν^7 + 199134540*ν^5 + 332549712*ν^3 + 71850240*ν
[1, 1, 2, 2, 2, 12]: 132*ν^14 + 14190*ν^12 + 539748*ν^10 + 9569505*ν^8 + 82096608*ν^6 + 299457180*ν^4 + 263051712*ν^2
[1, 1, 1, 1, 2, 2, 12]: 396*ν^13 + 53658*ν^11 + 1999008*ν^9 + 30418839*ν^7 + 190970406*ν^5 + 359436528*ν^3 + 71850240*ν


[1, 1, 1, 1, 1, 2, 5, 8]: 2100*ν^12 + 267750*ν^10 + 8370180*ν^8 + 94132815*ν^6 + 335543670*ν^4 + 216412560*ν^2
[1, 1, 1, 1, 1, 1, 1, 5, 8]: 14700*ν^11 + 1933050*ν^9 + 44635080*ν^7 + 263155725*ν^5 + 312331320*ν^3 + 32659200*ν
[4, 4, 4, 8]: 112*ν^14 + 10472*ν^12 + 413196*ν^10 + 8641766*ν^8 + 93753352*ν^6 + 334454337*ν^4 + 217455840*ν^2
[1, 3, 4, 4, 8]: 168*ν^13 + 19992*ν^11 + 1015854*ν^9 + 24780546*ν^7 + 199379523*ν^5 + 365666112*ν^3 + 63866880*ν
[2, 2, 4, 4, 8]: 56*ν^15 + 4984*ν^13 + 185066*ν^11 + 3635002*ν^9 + 38862845*ν^7 + 202706434*ν^5 + 345467808*ν^3 + 63866880*ν
[1, 1, 2, 4, 4, 8]: 56*ν^14 + 5880*ν^12 + 271194*ν^10 + 6687730*ν^8 + 78105909*ν^6 + 322948290*ν^4 + 246710016*ν^2
[1, 1, 1, 1, 4, 4, 8]: 168*ν^13 + 19992*ν^11 + 1015854*ν^9 + 24780546*ν^7 + 199379523*ν^5 + 365666112*ν^3 + 63866880*ν
[2, 3, 3, 4, 8]: 336*ν^13 + 40908*ν^11 + 1838718*ν^9 + 32396784*ν^7 + 210013251*ν^5 + 350926758*ν^3 + 59512320*ν
[1, 1, 3, 3, 4, 8]: 840*ν^12 + 110880*ν^10 + 5016270*ν^8 + 77478030*ν^6 + 33066

[1, 1, 1, 1, 1, 3, 6, 6]: 1125*ν^12 + 141000*ν^10 + 6976500*ν^8 + 94202250*ν^6 + 336069000*ν^4 + 217339200*ν^2
[2, 2, 2, 2, 6, 6]: 25*ν^16 + 2200*ν^14 + 80400*ν^12 + 1566770*ν^10 + 17301800*ν^8 + 104940600*ν^6 + 300045600*ν^4 + 230791680*ν^2
[1, 1, 2, 2, 2, 6, 6]: 25*ν^15 + 2500*ν^13 + 109500*ν^11 + 2522570*ν^9 + 30714200*ν^7 + 178527000*ν^5 + 357213600*ν^3 + 85639680*ν
[1, 1, 1, 1, 2, 2, 6, 6]: 75*ν^14 + 8250*ν^12 + 393900*ν^10 + 8707710*ν^8 + 86155380*ν^6 + 315919440*ν^4 + 243544320*ν^2
[1, 1, 1, 1, 1, 1, 2, 6, 6]: 375*ν^13 + 44250*ν^11 + 2166000*ν^9 + 38715750*ν^7 + 232433100*ν^5 + 337824000*ν^3 + 43545600*ν
[1, 1, 1, 1, 1, 1, 1, 1, 6, 6]: 2625*ν^12 + 325500*ν^10 + 15508500*ν^8 + 158202450*ν^6 + 364568400*ν^4 + 116121600*ν^2
[4, 5, 5, 6]: 1800*ν^12 + 198420*ν^10 + 7664850*ν^8 + 94640265*ν^6 + 333543450*ν^4 + 218680290*ν^2
[1, 3, 5, 5, 6]: 7200*ν^11 + 771450*ν^9 + 24752175*ν^7 + 199839300*ν^5 + 363757050*ν^3 + 65601900*ν
[2, 2, 5, 5, 6]: 900*ν^13 + 89400*ν^11 + 2832225*ν^9 + 37608000

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 4, 4]: 3780*ν^13 + 332640*ν^11 + 10679445*ν^9 + 125702010*ν^7 + 354715200*ν^5 + 163296000*ν^3
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4]: 41580*ν^12 + 3409560*ν^10 + 87952095*ν^8 + 343783440*ν^6 + 219542400*ν^4
[1, 3, 3, 3, 3, 3, 4]: 12960*ν^11 + 946080*ν^9 + 22742370*ν^7 + 201677445*ν^5 + 368114220*ν^3 + 61236000*ν
[2, 2, 3, 3, 3, 3, 4]: 864*ν^13 + 71712*ν^11 + 2273022*ν^9 + 33269751*ν^7 + 209244654*ν^5 + 351082512*ν^3 + 58786560*ν
[1, 1, 2, 3, 3, 3, 3, 4]: 3456*ν^12 + 262008*ν^10 + 6945858*ν^8 + 77519619*ν^6 + 323806086*ν^4 + 246192048*ν^2
[1, 1, 1, 1, 3, 3, 3, 3, 4]: 22032*ν^11 + 1467720*ν^9 + 28484946*ν^7 + 193637385*ν^5 + 362532672*ν^3 + 68584320*ν
[1, 2, 2, 2, 3, 3, 3, 4]: 216*ν^14 + 18306*ν^12 + 612387*ν^10 + 10180458*ν^8 + 85599828*ν^6 + 313809336*ν^4 + 244508544*ν^2
[1, 1, 1, 2, 2, 3, 3, 3, 4]: 972*ν^13 + 76896*ν^11 + 2246481*ν^9 + 30036798*ν^7 + 179244792*ν^5 + 364741056*ν^3 + 78382080*ν
[1, 1, 1, 1, 1, 2, 3, 3, 3, 4]: 6480*ν^12 + 461970*ν^10

ν

In [2]:
def deriv(d):
    sum=int()
    D=factor(d)
    fac=list(D)
    length=len(fac)
    for i in range(0, len(fac)):
        for j in range(1,(fac[i][1]+1)):
            sum+=(d/(fac[i][0]))
    return sum
deriv(2214502422)

2214502421